# Libraries

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Loading the dataset

In [86]:
df = pd.read_csv('/content/electricity.csv', header=0)
df.describe()
df.head()

,date,0,1,2,3,4,5,6,7,8,...,311,312,313,314,315,316,317,318,319,OT
0,2016-07-01 02:00:00,14.0,69.0,234.0,415.0,215.0,1056.0,29.0,840.0,226.0,...,676.0,372.0,80100.0,4719.0,5002.0,48.0,38.0,1558.0,182.0,2162.0
1,2016-07-01 03:00:00,18.0,92.0,312.0,556.0,292.0,1363.0,29.0,1102.0,271.0,...,805.0,452.0,95200.0,4643.0,6617.0,65.0,47.0,2177.0,253.0,2835.0
2,2016-07-01 04:00:00,21.0,96.0,312.0,560.0,272.0,1240.0,29.0,1025.0,270.0,...,817.0,430.0,96600.0,4285.0,6571.0,64.0,43.0,2193.0,218.0,2764.0
3,2016-07-01 05:00:00,20.0,92.0,312.0,443.0,213.0,845.0,24.0,833.0,179.0,...,801.0,291.0,94500.0,4222.0,6365.0,65.0,39.0,1315.0,195.0,2735.0
4,2016-07-01 06:00:00,22.0,91.0,312.0,346.0,190.0,647.0,16.0,733.0,186.0,...,807.0,279.0,91300.0,4116.0,6298.0,75.0,40.0,1378.0,191.0,2721.0


### Exploring the role of OT

In [87]:
# Exclude the 'date' column for correlation calculation
numeric_columns = df.select_dtypes(include=['float64', 'int64'])
correlations = numeric_columns.corr()['OT']  # Compute correlations with OT

# Display top and lowest correlations
print("Top Correlations with OT:")
print(correlations.sort_values(ascending=False).head(10))  # Top positive correlations
print("\nLowest Correlations with OT:")
print(correlations.sort_values().head(10))  # Top negative correlations

Top Correlations with OT:
OT     1.000000
119    0.709113
313    0.701160
117    0.673972
152    0.663269
278    0.655860
163    0.653981
175    0.650548
149    0.650249
167    0.646461
Name: OT, dtype: float64

Lowest Correlations with OT:
105   -0.508356
107   -0.461668
299   -0.376700
106   -0.355740
23    -0.103068
27    -0.086448
4     -0.055534
2     -0.047713
104   -0.047596
66    -0.040822
Name: OT, dtype: float64


Observations about the dataset :

--> columns :
- The first column (date): Timestamp of each observation at an hourly frequency.
- Columns 0 to 319: Hourly electricity consumption data for 320 entities.
- Column OT : seems to be an external variable, it could serve as an additional feature to help the model learn patterns in electricity consumption based on the correlation results above.

--> rows :  
- Each row represents one hour's data for all entities.

# Data Preparation

### Convert date to datetime (important step to work on time series)

In [88]:
# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Check for conversion issues (e.g., invalid dates)
invalid_dates = df['date'].isna().sum()
print(f"Number of invalid dates after conversion: {invalid_dates}")

Number of invalid dates after conversion: 0


### Check for Duplicates

In [89]:
# Check for duplicate timestamps
duplicate_dates = df['date'].duplicated().sum()
print(f"Number of duplicate timestamps: {duplicate_dates}")

Number of duplicate timestamps: 0


### Time continuity verification


In [90]:
# Calculate time differences between consecutive timestamps
time_diff = df['date'].diff().value_counts()
print("Time differences between rows:")
print(time_diff)

Time differences between rows:
date
0 days 01:00:00    26303
Name: count, dtype: int64


### Checking and handling missing values

In [91]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values[missing_values > 0])

Missing Values:
Series([], dtype: int64)


### Normalizing and standardizing features

In [92]:
# Normalize electricity consumption and OT
scaler = MinMaxScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])  # Exclude 'date' column

### Feature Engineering

In [93]:
# Extract time-based features depending on 2016 timetable
df['hour'] = df['date'].dt.hour
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month

In [94]:
df.head()

,date,0,1,2,3,4,5,6,7,8,...,314,315,316,317,318,319,OT,hour,day_of_week,month
0,2016-07-01 02:00:00,0.100000,0.233108,0.389351,0.354701,0.393053,0.532258,0.162921,0.399619,0.376667,...,0.165162,0.125906,0.036090,0.185366,0.350191,0.132944,0.358244,2,4,7
1,2016-07-01 03:00:00,0.128571,0.310811,0.519135,0.475214,0.533821,0.686996,0.162921,0.524263,0.451667,...,0.162502,0.166558,0.048872,0.229268,0.489323,0.184806,0.469760,3,4,7
2,2016-07-01 04:00:00,0.150000,0.324324,0.519135,0.478632,0.497258,0.625000,0.162921,0.487631,0.450000,...,0.149972,0.165400,0.048120,0.209756,0.492920,0.159240,0.457995,4,4,7
3,2016-07-01 05:00:00,0.142857,0.310811,0.519135,0.378632,0.389397,0.425907,0.134831,0.396289,0.298333,...,0.147767,0.160214,0.048872,0.190244,0.295572,0.142440,0.453190,5,4,7
4,2016-07-01 06:00:00,0.157143,0.307432,0.519135,0.295726,0.347349,0.326109,0.089888,0.348716,0.310000,...,0.144057,0.158528,0.056391,0.195122,0.309733,0.139518,0.450870,6,4,7


In [98]:
#Features extraction
customer_columns = [str(i) for i in range(320)]
external_columns = ['OT', 'hour', 'day_of_week', 'month']

### Splitting Data into training, validation and testing sets

In [99]:
# Define input features (X) and target variables (Y)
#X = df.drop(columns=['date'])
X = df[customer_columns + external_columns]
#Y = df['OT']  # Or other target variable
Y = df[customer_columns]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets (80% train, 20% validation)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Print out the shapes of each split
print(f"Training Set: {X_train.shape}, Validation Set: {X_val.shape}, Testing Set: {X_test.shape}")

Training Set: (16834, 324), Validation Set: (4209, 324), Testing Set: (5261, 324)


In [47]:
# Convert DataFrames to NumPy arrays
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_val = X_val.to_numpy()
Y_val = Y_val.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

### Preparing Data for TIme Series Modeling
Sequence preparation is necessary :    
- To Capture Temporal Dependencies as time series models learn patterns and dependencies between past and future values. This allows structuring the data into sequencies of input_steps (past observations) and output_steps (future predictions) as the model learns how the target variable evolves over time.
- To enable multi-step forecasting (predicting multiple time steps into the future)

=> THis slicing can be done in different ways such as TimeseriesGenerator and others but we choose the NumPy array slicing here because then the data prepared can be directly converted to PyTorch tensors using torch.from_numpy() since we are going to use PyTorch in order to create the NHits model.

In [101]:
# Parameters
input_steps = 24
output_steps = 12

# Prepare sequences for X_train, Y_train, X_val, Y_val, X_test, Y_test
def prepare_sequences(X, Y, input_steps, output_steps):
    n_samples = len(X) - input_steps - output_steps
    X_seq = np.array([X[i:i + input_steps] for i in range(n_samples)])
    Y_seq = np.array([Y[i + input_steps:i + input_steps + output_steps] for i in range(n_samples)])
    return X_seq, Y_seq

# Prepare sequences for train, validation, and test
X_train_seq, Y_train_seq = prepare_sequences(X_train, Y_train, input_steps, output_steps)
X_val_seq, Y_val_seq = prepare_sequences(X_val, Y_val, input_steps, output_steps)
X_test_seq, Y_test_seq = prepare_sequences(X_test, Y_test, input_steps, output_steps)

# Convert sequences to torch tensors for training
X_train_seq = torch.tensor(X_train_seq, dtype=torch.float32)
Y_train_seq = torch.tensor(Y_train_seq, dtype=torch.float32)
X_val_seq = torch.tensor(X_val_seq, dtype=torch.float32)
Y_val_seq = torch.tensor(Y_val_seq, dtype=torch.float32)
X_test_seq = torch.tensor(X_test_seq, dtype=torch.float32)
Y_test_seq = torch.tensor(Y_test_seq, dtype=torch.float32)

### Converting to PyTorch Dataset

In [102]:
# TimeSeriesDataset class remains the same
class TimeSeriesDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

### Creating datasets and dataloader

In [103]:
# Create datasets
train_dataset = TimeSeriesDataset(X_train_seq, Y_train_seq)
val_dataset = TimeSeriesDataset(X_val_seq, Y_val_seq)
test_dataset = TimeSeriesDataset(X_test_seq, Y_test_seq)

# Create DataLoader for train, validation, and test
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Example usage to check DataLoader outputs
for batch_X, batch_Y in train_dataloader:
    print(batch_X.shape, batch_Y.shape)
    break

torch.Size([32, 24, 324]) torch.Size([32, 12, 320])


### NHITS Implementation

I am going to follow the implementation plan below.

Step 1: Define the Building Blocks
- NHITS Block: A single stack.
- Residual Mechanism: Combines forecasts hierarchically.

Step 2: Assemble the Stacks
- Use multiple NHITS blocks in sequence.

Step 3: Combine Predictions
- Sum up predictions from all stacks to produce the final forecast.


#### NHITS Block definition
--> Each block is going to consist of:

- Linear layers to projects inputs and refine outputs.
- A mechanism to handle residuals for hierarchical forecasting.

In [66]:
"""class NHITSBlock(nn.Module):
    def __init__(self, input_steps, input_size, output_size, hidden_size):
        super(NHITSBlock, self).__init__()
        self.input_steps = input_steps
        self.input_size = input_size
        self.output_size = output_size

        # Fully connected layers
        self.fc1 = nn.Linear(input_steps * input_size, hidden_size)  # Flattened input
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Flatten the input: (batch_size, input_steps, input_size) -> (batch_size, input_steps * input_size)
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        output = self.fc_out(x)
        return output"""

#### Definition of the NHITS MOdel

This consists of combining multiple NHITS blocks and implementing the residual mechanism

In [ ]:
#Simple task model
"""class NHITSModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, input_steps, num_blocks):
        super(NHITSModel, self).__init__()
        self.num_blocks = num_blocks
        self.blocks = nn.ModuleList([
            NHITSBlock(input_size, output_size, hidden_size, input_steps)
            for _ in range(num_blocks)
        ])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)  # Forward pass through each NHITS block
        return x"""

In [104]:
#RESIDUAL MODEL
"""class NHITS(nn.Module):
    def __init__(self, input_steps, input_size, output_size, hidden_size, num_stacks):
        super(NHITS, self).__init__()
        self.num_stacks = num_stacks
        self.input_steps = input_steps
        self.input_size = input_size
        self.output_size = output_size

        # Create multiple NHITS blocks
        self.blocks = nn.ModuleList([
            NHITSBlock(input_steps, input_size, output_size, hidden_size) for _ in range(num_stacks)
        ])

        # Projection layer to map residuals back to input_size
        self.projection = nn.Linear(output_size, input_size)

    def forward(self, x):
        # Initialize residual as input
        residual = x
        forecasts = []

        for block in self.blocks:
            # Predict using the block
            forecast = block(residual)
            forecasts.append(forecast)

            # Update residual and project it back to input size
            residual_update = self.projection(forecast)

            # Expand the residual_update to match the shape of residual (broadcasting)
            residual_update_expanded = residual_update.unsqueeze(1).expand_as(residual)
            residual = residual - residual_update_expanded

        # Combine all forecasts
        final_forecast = sum(forecasts)
        return final_forecast"""

#### Initialising the Model

In [106]:
class NHITSBlock(nn.Module):
    def __init__(self, input_steps, input_size, output_size, hidden_size):
        super(NHITSBlock, self).__init__()
        self.input_steps = input_steps
        self.input_size = input_size
        self.output_size = output_size

        # Adding a recurrent layer
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size * input_steps, hidden_size)  # Flattened output from LSTM
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Pass through LSTM to capture temporal dependencies
        x, _ = self.rnn(x)  # Output shape: (batch_size, input_steps, hidden_size)

        # Flatten the output: (batch_size, input_steps, hidden_size) -> (batch_size, input_steps * hidden_size)
        x = x.contiguous().view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        output = self.fc_out(x)
        return output

In [114]:
class NHITS(nn.Module):
    def __init__(self, input_steps, input_size, output_size, hidden_size, num_stacks):
        super(NHITS, self).__init__()
        self.num_stacks = num_stacks
        self.input_steps = input_steps
        self.input_size = input_size
        self.output_size = output_size

        # Create multiple NHITS blocks
        self.blocks = nn.ModuleList([
            NHITSBlock(input_steps, input_size, output_size, hidden_size) for _ in range(num_stacks)
        ])

        # Projection layer to map residuals back to input_size
        self.projection = nn.Linear(output_size, input_size)

    def forward(self, x):
        # Initialize residual as input
        residual = x
        forecasts = []

        for block in self.blocks:
            # Predict using the block
            forecast = block(residual)
            forecasts.append(forecast)

            # Update residual and project it back to input size
            residual_update = self.projection(forecast)

            # Expand the residual_update to match the shape of residual (broadcasting)
            residual_update_expanded = residual_update.unsqueeze(1).expand_as(residual)
            residual = residual + residual_update_expanded  # Adding residual update to the residual

        # Combine all forecasts
        final_forecast = sum(forecasts)
        return final_forecast


In [115]:
# Parameters
input_steps = 24  # Number of past time steps
input_size = 324  # Number of features (320 customers + external features: 'OT', 'hour', 'day_of_week', 'month')
output_size = 320  # Forecast for 320 customers (same as the number of customers you're predicting)
hidden_size = 128  # Hidden layer size
num_stacks = 3     # Number of hierarchical blocks

# Create NHITS model
model = NHITS(input_steps, input_size, output_size, hidden_size, num_stacks)

# Example input
example_input = torch.randn(32, input_steps, input_size)  # 32 batches, 24 time steps, 324 features
output = model(example_input)
print(f"Output Shape: {output.shape}")  # Expected: (32, 320)

Output Shape: torch.Size([32, 320])


### Training and validation

#### Definition of the loss function and the optimizer

In [119]:
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

#### Training and validation

In [128]:
# Validation function
def validate(model, val_dataloader, criterion):
    model.eval()  # Set the model to evaluation mode (turn off dropout, batch norm)
    val_loss = 0.0  # Initialize validation loss
    with torch.no_grad():  # No need to compute gradients during validation
        for batch_X, batch_Y in val_dataloader:
            batch_Y = batch_Y.mean(dim=1)
            # Forward pass
            predictions = model(batch_X)

            # Compute loss
            loss = criterion(predictions, batch_Y)

            # Accumulate loss for this batch
            val_loss += loss.item()

    # Return the average validation loss
    avg_val_loss = val_loss / len(val_dataloader)
    return avg_val_loss


In [129]:
# Parameters
num_epochs = 20  # Number of epochs to train
batch_size = 32  # Batch size

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for batch_X, batch_Y in train_dataloader:

        batch_Y = batch_Y.mean(dim=1)

        # Print shapes for debugging
        #print(f"batch_X shape: {batch_X.shape}")
        #print(f"batch_Y shape: {batch_Y.shape}")

        # Forward pass
        predictions = model(batch_X)

        # Print prediction shape for debugging
        #print(f"Predictions shape: {predictions.shape}")


        # Compute loss
        loss = criterion(predictions, batch_Y)

        # Backward pass (compute gradients)
        optimizer.zero_grad()  # Zero out previous gradients

        loss.backward()

        # Update weights
        optimizer.step()

        # Accumulate loss for this batch
        running_loss += loss.item()

    # Print the average loss for this epoch
    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:}")

    # Perform validation after each epoch
    val_loss = validate(model, val_dataloader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:}")


Epoch [1/20], Loss: 0.0008023441891141591
Epoch [1/20], Validation Loss: 0.0030719717033207417
Epoch [2/20], Loss: 0.0007843783213978721
Epoch [2/20], Validation Loss: 0.003007119852971314
Epoch [3/20], Loss: 0.0007694837947686513
Epoch [3/20], Validation Loss: 0.0030827825704148708


KeyboardInterrupt: 

#### Validation